In [2]:
import minerl

download_data = False

if download_data:
    import os
    data_path = 'C:\temp\data'
    os.environ["MINERL_DATA_ROOT"] = data_path
    minerl.data.download(experiment="MineRLTreechopVectorObf-v0")    

C:\Users\ngune07417\AppData\Roaming\Python\Python37\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
from sklearn.cluster import KMeans
import tqdm
import numpy as np

dat = minerl.data.make('MineRLTreechopVectorObf-v0')

# Load the dataset storing 1000 batches of actions
act_vectors = []
for _, act, _, _,_ in tqdm.tqdm(dat.batch_iter(16, 32, 2, preload_buffer_size=20)):
    act_vectors.append(act['vector'])
    if len(act_vectors) > 1000:
        break

# Reshape these the action batches
acts = np.concatenate(act_vectors).reshape(-1, 64)
kmeans_acts = acts[:100000]

# Use sklearn to cluster the demonstrated actions
kmeans = KMeans(n_clusters=32, random_state=0).fit(kmeans_acts)

0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 209/209 [00:00<00:00, 691.36it/s]
755it [01:06, 67.93it/s]Exception in thread QueueManagerThread:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\MineRL\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\envs\MineRL\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\ProgramData\Anaconda3\envs\MineRL\lib\concurrent\futures\process.py", line 361, in _queue_management_worker
    ready = wait(readers + worker_sentinels)
  File "C:\ProgramData\Anaconda3\envs\MineRL\lib\multiprocessing\connection.py", line 873, in wait
    ov.cancel()
OSError: [WinError 6] The handle is invalid

838it [01:07, 60.03it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████

In [14]:
import gym

NUM_CLUSTERS = 32
i, net_reward, done = 0, 0, False

if 'env' not in locals():
    env = gym.make('MineRLTreechopVectorObf-v0')

obs = env.reset()

while not done:
    # Let's use a frame skip of 4 (could you do better than a hard-coded frame skip?)
    if i % 4 == 0:
        action = {
            'vector': kmeans.cluster_centers_[np.random.choice(NUM_CLUSTERS)]
        }

    obs, reward, done, info = env.step(action)
    #env.render()

    if reward > 0:
        print("+{} reward!".format(reward))
    net_reward += reward
    i += 1

print("Total reward: ", net_reward)

Total reward:  0.0
